In [0]:
from pyspark.sql.functions import col
from pyspark.sql.types import IntegerType, DoubleType, BooleanType, DateType

In [0]:
configs = {"fs.azure.account.auth.type": "OAuth",
"fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
"fs.azure.account.oauth2.client.id": "client.id",
"fs.azure.account.oauth2.client.secret": "client.secret",
"fs.azure.account.oauth2.client.endpoint": "https://login.microsoftonline.com/client.endpoint/oauth2/token"}
 
dbutils.fs.mount(
source = "abfss://container.name@storage.name.dfs.core.windows.net", # container@storageacc
mount_point = "/mnt/deproject",
extra_configs = configs)

True

In [0]:
%fs
ls "/mnt/deproject"

path,name,size,modificationTime
dbfs:/mnt/deproject/raw-data/,raw-data/,0,1699316986000
dbfs:/mnt/deproject/transformed-data/,transformed-data/,0,1699316999000


In [0]:
athletes = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/deproject/raw-data/athletes.csv")
coaches = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/deproject/raw-data/coaches.csv")
gender = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/deproject/raw-data/entriesgender.csv")
medals = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/deproject/raw-data/medals.csv")
teams = spark.read.format("csv").option("header","true").option("inferSchema","true").load("/mnt/deproject/raw-data/teams.csv")

In [0]:
gender.printSchema();

root
 |-- Discipline: string (nullable = true)
 |-- Female: integer (nullable = true)
 |-- Male: integer (nullable = true)
 |-- Total: integer (nullable = true)



In [0]:
#Find the top countries with the hiughest number of gold medals

top_gold_medals = medals.orderBy("Gold",ascending=False).select("Team_Country","Gold");
top_gold_medals.show();

+--------------------+----+
|        Team_Country|Gold|
+--------------------+----+
|United States of ...|  39|
|People's Republic...|  38|
|               Japan|  27|
|       Great Britain|  22|
|                 ROC|  20|
|           Australia|  17|
|         Netherlands|  10|
|              France|  10|
|             Germany|  10|
|               Italy|  10|
|                Cuba|   7|
|         New Zealand|   7|
|              Brazil|   7|
|              Canada|   7|
|             Hungary|   6|
|   Republic of Korea|   6|
|               Kenya|   4|
|              Poland|   4|
|      Czech Republic|   4|
|              Norway|   4|
+--------------------+----+
only showing top 20 rows



In [0]:
athletes.write.mode("overwrite").option("header","true").csv("/mnt/deproject/transformed-data/athletes")
coaches.write.mode("overwrite").option("header","true").csv("/mnt/deproject/transformed-data/coaches")
gender.write.mode("overwrite").option("header","true").csv("/mnt/deproject/transformed-data/gender")
medals.write.mode("overwrite").option("header","true").csv("/mnt/deproject/transformed-data/medals")
teams.write.mode("overwrite").option("header","true").csv("/mnt/deproject/transformed-data/teams")